In [23]:
#!pip uninstall -y transformers && pip install git+https://github.com/huggingface/transformers

In [24]:
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [25]:
#!pip install -q --upgrade bitsandbytes  peft trl

In [26]:
#!pip uninstall -y accelerate && pip install accelerate

In [4]:
#!pip install git+https://github.com/huggingface/peft

In [27]:
#!pip install --upgrade peft

In [1]:
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [28]:
#!pip install tensorboardX

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
import pandas as pd
import os
import numpy as np
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
from datasets import Dataset

/opt/conda/envs/python39/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
os.getcwd()

'/home/jupyter/finetune_LLM'

In [4]:
classified_krishna_df = pd.read_csv('/home/jupyter/finetune_LLM/Data/train/processed_chunked_files_llm_finetuining_krishna.csv') #pd.read_csv('/home/necuser/sdp/Classification/LLM/Finetuining/data/train/processed_chunked_files_llm_finetuining_krishna.csv')
classified_sandhya_df = pd.read_csv('/home/jupyter/finetune_LLM/Data/train/processed_chunked_files_llm_finetuining_sandhya.csv') #pd.read_csv('/home/necuser/sdp/Classification/LLM/Finetuining/data/train/processed_chunked_files_llm_finetuining_sandhya.csv')

# Concatenating the DataFrames row-wise
classified_df = pd.concat([classified_sandhya_df, classified_krishna_df], axis=0, ignore_index=True)
classified_df['notes_character_count'] = classified_df['notes'].str.len()

from transformers import AutoTokenizer
max_length = 2048 #1024 #20000 # This was an appropriate max length for my dataset

# redefine the tokenize function and tokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "BioMistral/BioMistral-7B",
)
#tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"

def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

classified_df['token_count_notes'] = classified_df['notes'].apply(lambda x: len(tokenizer.encode(x)))
classified_df['token_count_classified'] = classified_df['classified'].apply(lambda x: len(tokenizer.encode(x)))


dataset = Dataset.from_pandas(classified_df)
dataset = dataset.train_test_split(0.1)

In [5]:
from datetime import datetime
from functools import partial

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from trl import  SFTTrainer, DataCollatorForCompletionOnlyLM
from peft import LoraConfig, PeftModel, get_peft_model


In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['filename', 'notes', 'classified', 'notes_character_count', 'token_count_notes', 'token_count_classified'],
        num_rows: 535
    })
    test: Dataset({
        features: ['filename', 'notes', 'classified', 'notes_character_count', 'token_count_notes', 'token_count_classified'],
        num_rows: 60
    })
})

In [15]:
class config:
    #
    base_model = "BioMistral/BioMistral-7B"
    #
    lora_r = 16
    lora_alpha = 64
    lora_dropout = 0.05
    lora_target_modules = [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ]

    #
    use_4bit = True
    use_8bit = False
    bnb_4bit_compute_dtype = "float16"
    bnb_4bit_quant_type = "nf4"
    bnb_4bit_use_double_quant = (
        False
    )

    #
    run_name = "snp-finetune-classifier-2024_300624_v1"
    output_dir = "/home/jupyter/finetune_LLM/finetuned_LLM/"+run_name #f"./results_llm/{project_name}"
    num_train_epochs = 3
    fp16 = True
    bf16 = False
    per_device_train_batch_size = 4
    gradient_accumulation_steps = (
        128
    )

    gradient_checkpointing = False
    max_grad_norm = 1.0
    learning_rate = 5e-5
    weight_decay = (
        0.001
    )
    optim = "paged_adamw_32bit"
    adam_beta1 = 0.9
    adam_beta2 = 0.999
    adam_epsilon = 1e-8
    lr_scheduler_type = "cosine"
    max_steps = -1
    warmup_ratio = 0.03
    group_by_length = False
    save_steps = 100
    logging_steps = 5

    #
    max_seq_length = 2048
    packing = False
    device_map = {"": 0}
    NUM_CORES = 10
    use_flash_attn = False


In [8]:
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    config.base_model,
    trust_remote_code=True,
    truncation=True,
    padding=True

)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

In [9]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example["notes"])):
        text = f"""Classify the notes into below categories:
                   'allergies',
                   'chief_complaints',
                   'diagnosis',
                   'family_history',
                   'history',
                   'instructions_advice',
                   'investigation_report',
                   'investigations',
                   'medicine_prescription',
                   'observations_examinations',
                   'patient_willingness_concent',
                   'personal_history',
                   'procedure_report',
                   'referral',
                   'social_history',
                   'tolerance',
                   'treatment_plan',
                   'unclassified',
                   'vitals'
                    Output the classified data into json format


        ### notes:
        {example["notes"]}

        ### Classified:
        {example["classified"]}
        """
        output_texts.append(text)

    return output_texts

response_template = " ### Classified:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)


In [10]:
collator

DataCollatorForCompletionOnlyLM(tokenizer=LlamaTokenizerFast(name_or_path='BioMistral/BioMistral-7B', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, mlm=False, mlm_probability=0.15, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt')

In [11]:
config.use_4bit

True

In [12]:
import torch
# BitsAndBytes config ( model quantisation config )
if config.use_4bit:
    bnb_config = BitsAndBytesConfig(
        load_in_8bit=config.use_8bit,
        load_in_4bit=config.use_4bit,
        bnb_4bit_quant_type=config.bnb_4bit_quant_type,
        bnb_4bit_compute_dtype=config.bnb_4bit_compute_dtype,
        bnb_4bit_use_double_quant=config.bnb_4bit_use_double_quant,
    )
elif config.use_8bit:
    bnb_config = BitsAndBytesConfig(
        load_in_8bit=config.use_8bit,
    )
else:
    assert True, "Set one of 'use_4bit' or 'use_8bit' to True!"

# load base model
model = AutoModelForCausalLM.from_pretrained(
    config.base_model,
    quantization_config=bnb_config,
    device_map=config.device_map,
    attn_implementation=("flash_attention_2" if config.use_flash_attn else "eager"),
    low_cpu_mem_usage=True
)
model.gradient_checkpointing_enable(
    gradient_checkpointing_kwargs={"use_reentrant": False}
)
model.enable_input_require_grads()
model.config.use_cache = False
model.train()

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
   

In [16]:
# load LoRA configuration
lora_config = LoraConfig(
    lora_alpha=config.lora_r * 2,
    lora_dropout=config.lora_dropout,
    r=config.lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=config.lora_target_modules,
)

# attach LoRA adapter to model
model = get_peft_model(model, lora_config)

# for logging purposes
current_datetime = datetime.now().strftime("%Y-%m-%d-%H-%M")
working_dir = f"{config.output_dir}_{current_datetime}"



In [17]:
working_dir

'/home/jupyter/finetune_LLM/finetuned_LLM/snp-finetune-classifier-2024_300624_v1_2024-06-30-18-34'

In [18]:
dataset

DatasetDict({
    train: Dataset({
        features: ['filename', 'notes', 'classified', 'notes_character_count', 'token_count_notes', 'token_count_classified'],
        num_rows: 535
    })
    test: Dataset({
        features: ['filename', 'notes', 'classified', 'notes_character_count', 'token_count_notes', 'token_count_classified'],
        num_rows: 60
    })
})

In [21]:
dataset["train"]

Dataset({
    features: ['filename', 'notes', 'classified', 'notes_character_count', 'token_count_notes', 'token_count_classified'],
    num_rows: 535
})

In [30]:
# set training parameters
training_arguments = TrainingArguments(
    output_dir=working_dir,
    num_train_epochs=config.num_train_epochs,
    per_device_train_batch_size=config.per_device_train_batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    optim=config.optim,
    save_steps=config.save_steps,
    logging_steps=config.logging_steps,
    learning_rate=config.learning_rate,
    weight_decay=config.weight_decay,
    fp16=config.fp16,
    bf16=config.bf16,
    max_grad_norm=config.max_grad_norm,
    max_steps=config.max_steps,
    warmup_ratio=config.warmup_ratio,
    group_by_length=config.group_by_length,
    lr_scheduler_type=config.lr_scheduler_type,
    adam_beta1=config.adam_beta1,
    adam_beta2=config.adam_beta2,
    adam_epsilon=config.adam_epsilon,
    logging_dir=f"{working_dir}/logs",
    #report_to="tensorboard",
    remove_unused_columns = ["filename", "notes_character_count", "token_count_notes", "token_count_classified"]
)

trainer = SFTTrainer(
    model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    args=training_arguments,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    max_seq_length=config.max_seq_length
)


/opt/conda/envs/python39/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/envs/python39/lib/python3.10/site-packages/transformers/training_args.py:1961: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/opt/conda/envs/python39/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
Map: 100%|██████████| 60/60 [00:01<00:00, 45.27 examples/s]
/opt/conda/envs/python39/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:397: UserWarning: 

In [31]:
trainer.train()

/opt/conda/envs/python39/lib/python3.10/site-packages/trl/trainer/utils.py:187: UserWarning: Could not find response key ` ### Classified:` in the following instance: <s> Classify the notes into below categories:
                   'allergies',
                   'chief_complaints',
                   'diagnosis',
                   'family_history',
                   'history',
                   'instructions_advice',
                   'investigation_report',
                   'investigations',
                   'medicine_prescription',
                   'observations_examinations',
                   'patient_willingness_concent',
                   'personal_history',
                   'procedure_report',
                   'referral',
                   'social_history',
                   'tolerance',
                   'treatment_plan',
                   'unclassified',
                   'vitals'
                    Output the classified data into json format


        #

Step,Training Loss


TrainOutput(global_step=3, training_loss=0.0, metrics={'train_runtime': 1530.143, 'train_samples_per_second': 1.049, 'train_steps_per_second': 0.002, 'total_flos': 1.3482508137214771e+17, 'train_loss': 0.0, 'epoch': 2.8656716417910446})

In [32]:
# save trained model
trainer.model.save_pretrained(f"{working_dir}/checkpoint-final")